# ETL-процесс и визуализация по данным о фильмах

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Источник данных (Extract)

Пять CSV-файлов:
* **film_work.csv** — фильмы (id, title, creation_date, rating, type, и др.)
* **genre.csv** — жанры (id, name)
* **genre_film_work.csv** — связи фильмов и жанров (film_work_id, genre_id)
* **person.csv** — участники (id, full_name)
* **person_film_work.csv** — связи фильмов и персон (film_work_id, person_id, role)


In [ ]:
film_work        = 'data/raw/content_film_work.xlsx'
genre            = 'data/raw/content_genre.xlsx'
genre_film_work  = 'data/raw/content_genre_film_work.xlsx'
person           = 'data/raw/content_person.xlsx'
person_film_work = 'data/raw/content_person_film_work.xlsx'

In [ ]:
df_fw   = pd.read_excel(film_work)
df_g    = pd.read_excel(genre)
df_gfw  = pd.read_excel(genre_film_work)
df_p    = pd.read_excel(person)
df_pfw  = pd.read_excel(person_film_work)

In [ ]:
tables = [df_fw, df_g, df_gfw, df_p, df_pfw]

## 2. Трансформация данных (Transform)

### 2.1. Очистка данных
* Преобразовать creation_date в тип datetime, оставить только год.
* Убедиться, что rating — числовой (float), обработать пустые/некорректные значения.
* Проверить дубликаты, особенно в таблицах связей (многие-ко-многим).

#### 2.1.1. Выводы
На основе вывода .info() ниже получили следующие выводы:
1. **creation_date** польностью нулевой, поэтому мы заполним его значениями из колонки **created**, предварительно конвертировав обе колонки в *datetime*.
2. **rating** имеет тип object и данные выглядят в форме даты, поэтому пришлось переводить первоначально в datetime, а потом извлекать из него рейтинг

In [ ]:
df_fw.info()

In [ ]:
# 1. Поменяем тип для creation_date, сделав его datetime, 
#    передав туда created в формате datetime
df_fw['creation_date'] = pd.to_datetime(
    pd.to_datetime(df_fw['created'], errors='coerce')
)

# 2. Оставим только год
df_fw['creation_date'] = df_fw['creation_date'].dt.year

In [ ]:
# 1. Поменяем предварительно тип для rating на datetime
df_fw['rating'] = pd.to_datetime(df_fw['rating'], errors='coerce')

# 2. Видно, что рейтинг, вида 8.6 записан в формате year-06-08, 
#    поэтому извлекаем день + месяц / 10
df_fw['rating'] = df_fw['rating'].dt.day + df_fw['rating'].dt.month / 10

# 3. Заполняем пропуски средней оценкей по всей выборке
df_fw['rating'] = df_fw['rating'].fillna(df_fw['rating'].mean())

In [ ]:
print("Количество дубликатов:")

for df in [df_fw, df_g, df_gfw, df_p, df_pfw]:
    print(df.duplicated().sum())
    
    df.drop_duplicates(inplace=True)